In [ ]:
import json
import pandas as pd

# Cargar los diccionarios desde un archivo JSON
with open("../../assets/grupos_farmacologicos.json", "r", encoding="utf-8") as f:
    diccionarios = json.load(f)

# Obtener los diccionarios
grupo_anatomico_dict = diccionarios["grupo_anatomico"]
subgrupo_terapeutico_dict = diccionarios["subgrupo_terapeutico"]
subgrupo_farmacologico_dict = diccionarios["subgrupo_farmacologico"]


# Función para mapear el código ATC
def mapear_atc(codigo_atc):
    if codigo_atc:  # Verifica si la cadena no está vacía
        grupo_anat = grupo_anatomico_dict.get(codigo_atc[0], "Desconocido")
        subgr_ter = subgrupo_terapeutico_dict.get(codigo_atc[:3], "Desconocido")
        subgr_farma = subgrupo_farmacologico_dict.get(codigo_atc[:4], "Desconocido")
    else:
        grupo_anat = "Desconocido"
        subgr_ter = "Desconocido"
        subgr_farma = "Desconocido"
    return grupo_anat, subgr_ter, subgr_farma


# Cargar el archivo JSON con las fichas técnicas
medicamentos_df = pd.read_json(
    "../../data/outputs/1_data_acquisition/wrangler/fichas_tecnicas.json"
)

# Eliminar duplicados
medicamentos_df = medicamentos_df.drop_duplicates()

# Llenar valores nulos en las columnas específicas
medicamentos_df = medicamentos_df.fillna(
    {"codigo_atc": "Desconocido", "indicaciones_terapeuticas": "Desconocido"}
)

# Verificar que no haya valores nulos en 'codigo_atc'
if medicamentos_df["codigo_atc"].isnull().any():
    print("Hay valores nulos en 'codigo_atc', se van a llenar con 'Desconocido'.")

# Aplicar la función de mapeo a la columna 'codigo_atc'
mapeo_atc = medicamentos_df["codigo_atc"].apply(mapear_atc)

# Crear nuevas columnas en el DataFrame con los resultados del mapeo
medicamentos_df[
    ["grupo_anatomico", "subgrupo_terapeutico", "subgrupo_farmacologico"]
] = pd.DataFrame(mapeo_atc.tolist(), index=medicamentos_df.index)

# Verificar cómo quedó el DataFrame
display(
    medicamentos_df[
        [
            "codigo_atc",
            "grupo_anatomico",
            "subgrupo_terapeutico",
            "subgrupo_farmacologico",
        ]
    ].head()
)

# Guardar el DataFrame con las nuevas columnas mapeadas en un nuevo archivo JSON (formato legible)
medicamentos_df.to_json(
    "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json",
    orient="records",
    lines=False,
    force_ascii=False,
    indent=4,
)

,codigo_atc,grupo_anatomico,subgrupo_terapeutico,subgrupo_farmacologico
0,No especificado,Sistema nervioso,Desconocido,Desconocido
1,N01AX63,Sistema nervioso,Desconocido,Desconocido
2,A11GA01.,Tracto alimentario y metabolismo,Desconocido,Desconocido
3,D06BB03,Dermatológicos,Desconocido,Desconocido
4,,Desconocido,Desconocido,Desconocido
